## 망고플레이트 키워드 맛집 크롤링

## 1page만 크롤링

In [12]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 지역 키워드를 입력하기
keyword = '경복궁'

# page 정하기
page = 1

In [28]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.maximize_window() # 창 최대화

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

# 방법5
# 열려 있는 세션 개수 확인
popup_count = len(driver.window_handles)
i = popup_count

# 팝업창 제거
while i > 0:
    if i == 1:
        driver.switch_to.window(driver.window_handles[i-1])
        i = 0
        break
    else:
        driver.switch_to.window(driver.window_handles[i-1])
        driver.close()
        i -= 1
        
# 현재 열려 있는 세션 확인
print(driver.window_handles)

['1F01A1A74A1032077AA92571CEF72FE7']


In [29]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 200)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element))).click()

            # 가게 이름 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
                title_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                title = title_raw.text
            except:
                pass
            
            # 가게 주소 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(1) > td'
                address_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                address = address_raw.text
            except:
                pass
            
            # 가게 영업시간 정보 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(6) > td'
                time_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                time = time_raw.text
            except:
                pass
            
            # 가게 마지막 주문 정보 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(7) > td'
                lastOrder_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                lastOrder = lastOrder_raw.text
            except:
                pass
            
            # 가게 휴일 정보 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(8) > td'
                holiday_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                holiday = holiday_raw.text
            except:
                pass
            
            # 가게 전체 평점 점수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
                total_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                total = total_raw.text
            except:
                pass
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element)))
                num_star = star_raw.text

            # 음식 종류 및 메뉴 크롤링
            try:
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
                type_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element1)))
                type_raw = type_raw.text
                food_type = type_raw
            except:
                element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(2) > td > span'
                type_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element1)))
                type_raw = type_raw.text
                food_type = type_raw
            try:
                element2 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(9) > td > ul'
                menu_raw = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, element2)))
                food_menu = menu_raw.text
                food_type = [type_raw, food_menu]
            except:
                pass
            
            # sub_dict에 담기
            sub_dict['title'] = title # 가게 이름
            
            sub_dict['Address'] = address  # 주소
            sub_dict['Time'] = time     # 영업시간
            sub_dict['LastOrder'] = lastOrder # 마지막 주문
            sub_dict['Holiday'] = holiday  # 휴일
            
            sub_dict['Point'] = total # 별점
            sub_dict['View'] = view   # 조회수
            sub_dict['Review'] = num_review # 리뷰개수
            sub_dict['Star'] = num_star   # 별표개수
            sub_dict['Type'] = food_type  # 음식 종류 및 메뉴
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            
            # 뒤로 가기
            driver.back()
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    driver.close()
    return df

In [30]:
df = FoodCrawling(row=10, col=2)
df

,title,Address,Time,LastOrder,Holiday,Point,View,Review,Star,Type
0,BAR CHAM,서울특별시 종로구 자하문로7길 34\n지번 서울시 종로구 통인동 135-5,16:00 - 22:00,21:30,화,4.8,"111,167",41,"4,047",칵테일 / 와인
1,우육면관,서울특별시 종로구 종로7길 29-14\n지번 서울시 종로구 청진동 39,11:00 - 21:00,월-금: 14:30 - 17:30\n토: 14:30 - 17:00,"14:30, 20:00",4.8,"46,481",43,"1,140",기타 중식
2,모던샤브하우스,서울특별시 종로구 종로3길 17 D타워 리플레이스 광화문 5F\n지번 서울시 종로구...,11:30 - 22:00,21:00,식당 홈페이지로 가기,4.8,"74,086",35,"1,129","[탕 / 찌개 / 전골, 시그니쳐 (성인)\n53,000원\n스페셜 (성인)\n78..."
3,만가타,서울특별시 종로구 삼청로2길 40-5\n지번 서울시 종로구 소격동 88-17,11:30 - 21:30,"월, 화",식당 홈페이지로 가기,4.7,"192,290",55,"4,210","[세계음식 기타, 클래식 코스\n59,000원\n테이스팅 코스\n69,000원\n미..."
4,스시소라,서울특별시 종로구 종로 19 르메이에르종로타운 B동 B116 B-1호\n지번 서울시...,11:30 - 21:00,14:30 - 16:30,식당 홈페이지로 가기,4.6,"296,946",75,"5,450","[정통 일식 / 일반 일식, 런치 카운터 스시 오마카세\n55,000원\n디너 카운..."
5,팔마,서울특별시 종로구 자하문로5길 39 1F\n지번 서울시 종로구 필운동 170 1F,17:00 - 24:00,월,식당 홈페이지로 가기,4.6,"111,639",47,"3,514","[퓨전 양식, 까르니타스\n7,000원\n돼지갈비 바르바코아\n7,000원\n프로슈..."
6,폴앤폴리나,서울특별시 종로구 사직로8길 42 1F\n지번 서울시 종로구 내수동 74 1F,12:00 - 19:00,"일, 첫째 월","바게트\n4,500원\n화이트바게트\n4,500원\n버터프레첼\n3,300원\n플레...",4.6,"380,954",163,"6,212",베이커리
7,물랑,서울특별시 종로구 자하문로16길 8 1F\n지번 서울시 종로구 창성동 87-1 1F,12:00 - 22:00,15:00 - 18:00,수,4.6,"84,298",32,"1,912",프랑스 음식
8,아키비스트,서울특별시 종로구 효자로13길 52 1F\n지번 서울시 종로구 효자동 164-7 1F,11:00 - 21:00,식당 홈페이지로 가기,"아인슈페너\n6,000원\n아메리카노\n5,000원\n카페라떼\n5,500원\n아몬...",4.6,"39,649",55,"1,933",카페 / 디저트
9,호가양꼬치,서울특별시 종로구 율곡로4길 66\n지번 서울시 종로구 수송동 71-1,월-금: 13:00 - 24:00\n토-일: 13:00 - 22:30,"양갈비 (300g)\n28,000원\n양꼬치 (200g)\n14,000원\n탕수육\...",+ 7,4.5,"213,911",83,"4,248",기타 중식


In [31]:
df.to_csv('경복궁_추가.csv', index=False, encoding='utf-8-sig')